# Simple 3D Finite Elasticity model

This is simple Elasticity Model in 3D. The model is implemented in Python using the FEniCS Project.

In [1]:
import numpy as np

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, Expression )
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import XDMFFile
import ufl
from ufl import (TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional , gt, dx, inner, derivative, dot, ln, split,TestFunction)
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista
pyvista.set_jupyter_backend('client')
## Define temporal parameters

## Some parameters

In [2]:
e_dot = 0.001 # strain rate per second 
e_max = 0.01 # absolute value of maximum strain


E = 200e9 # Young’s modulus 
Y0 = 100e6 # Initial yield 
h0 = 0 # Initial hardening modulus 
Y_star = 100e6 # Yield saturation 
r = 0 #Hardening exponent 
C = 0 #Backstress modulus 
gamma = 0 # Nonlinear kinematic parameter



# Define Geometry

In [3]:
length = 10.0 # mm
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0],[length,length,length]],[2,2,2],mesh.CellType.tetrahedron)

In [4]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:63921/index.html?ui=P_0x28a3b3c10_0&reconnect=auto' style='width: …

# Definig Function Spaces


In [11]:
# Tensor space
T_Ele = ufl.TensorElement("Quadrature", domain.ufl_cell(),1,symmetry=True)
U_Ele = ufl.VectorElement("CG", domain.ufl_cell(),1)

T_quad = FunctionSpace(domain,T_Ele)
V_u = FunctionSpace(domain,U_Ele)

In [6]:
#MAT Properties

K = Constant(domain,200.0e6) #200 GPa Bulk Modulus
G = Constant(domain,80.0e6) #80 GPa Shear Modulus

Lambda = 3

I = np.eye(3)


E = np.zeros((3,3))

#Fourth Order Symmetric Identity Matrix
Ident_sym = (np.einsum('ik,jl->ijkl',I,I) + np.einsum('il,jk->ijkl',I,I))/2


C = 2*G*Ident_sym + (K-(2/3)*G)*np.tensordot(I,I,0)


In [13]:
u = TrialFunction(V_u)

Transposed(Grad(Argument(FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False), (3,)), 0), VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3)), 1, None)))

In [28]:
E_expr = Expression(1/2*(grad(u)+(grad(u).T)),V_u.element.interpolation_points())

In [31]:
E = Function(T_quad)
E.interpolate(E_expr)

RuntimeError: Cannot interpolate Expression with Argument